In [1]:
# imports

import zlib
import lzma
import bz2
import os
import io
import sys
import time
import csv
import zstandard as zstd
import blosc as bl
import pickle
import re
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
class Genome:
    def __init__()

SyntaxError: invalid syntax (<ipython-input-2-38fb9df2b9d8>, line 2)

In [3]:
# decorator to auth for google drive -- see: https://developers.google.com/drive/api/v3/quickstart/python
def auth_drive(func):
    def wrapper(*args, **kwargs):
        creds = None
        SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server()
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        service =  build('drive', 'v2', credentials=creds)
        kwargs['service'] = service
        return func(*args, **kwargs)
            
    return wrapper

# generator that obtains and collates files
@auth_drive
def gen_links(service):
    results = service.children().list(folderId='1mHg9cmN6GrjEQCK02c_nd-FUHODbZ4Z5').execute()
    items = results.get('items', [])
    for item in items:
        print('generating item link')
        yield item['id']
        
@auth_drive
def gen_files(file_ids, service):
    for file_id in file_ids:
        result = service.files().get(fileId=file_id).execute()
        if re.search('genome\.(gz)$', result['title']):
            request = service.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print("Download %d%%." % int(status.progress() * 100))
                
            # construct object with metadata?
            yield {'id': file_id, 'file': fh}

@auth_drive
def test(service):
    results = service.children().list(folderId='1mHg9cmN6GrjEQCK02c_nd-FUHODbZ4Z5', fields="nextPageToken, files(id, title)").execute()
    items = results.get('items', [])
    for item in items:
        print(item)
        #print('File Id: %s, NameL %s' % (item['id'], item['title']))

    
        
            

In [ ]:
list(gen_files(gen_links()))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=688169560602-hbljcplp74iclbus423nt38hm6e7ru5g.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=Kj21MwWH6LfTrcSKy9nKDqATz1XKBQ&access_type=offline
generating item link
generating item link
Download 54%.
Download 100%.
generating item link
Download 100%.
generating item link
Download 100%.
generating item link
Download 100%.
generating item link
Download 100%.
generating item link
Download 19%.
Download 39%.
Download 59%.
Download 79%.


In [65]:
a.to_json()

'{"uri": "https://www.googleapis.com/drive/v2/files/1IxTEzuRJ--6r10kudtSo3VweJ7PrhyLHzY1T1FwbChs?alt=media", "method": "GET", "body": null, "headers": {"accept": "*/*", "accept-encoding": "gzip, deflate", "user-agent": "google-api-python-client/1.7.8 (gzip)"}, "methodId": "drive.files.get", "resumable": null, "response_callbacks": [], "_in_error_state": false, "body_size": 0, "resumable_uri": null, "resumable_progress": 0}'

In [7]:
# compression transforms
# binary file --> binary compressed file

def comp_ratio(compressor):
    def wrapper(*args, **kwargs):
        uncompressed_size = sys.getsizeof(args[0])
        compressed = compressor(*args, **kwargs)
        compressed_size = sys.getsizeof(compressed)
        return uncompressed_size / compressed_size
    return wrapper

@comp_ratio
def zlib_compress(f):
    c_z = zlib.compress(f,9)
    return c_z

@comp_ratio
def lzma_compress(f):
    c = lzma.LZMACompressor()
    c_l = c.compress(f)
    c_l += c.flush()
    return c_l

@comp_ratio
def bz2_compress(f):
    c = bz2.BZ2Compressor()
    c_b = c.compress(f)
    c_b += c.flush()
    return c_b

@comp_ratio
def zstd_compress(f):
    c = zstd.ZstdCompressor()
    c_zs = c.compress(f)
    return c_zs

@comp_ratio
def bl_compress(f):
    return bl.compress(f)

compress_transforms = [zlib_compress, lzma_compress, bz2_compress, zstd_compress, bl_compress]

In [48]:
#generator that applies list of transforms to list of files
def apply(genomes, transforms, log='results.csv'):
    with open(log, 'a', newline='') as outfile:
        writer = csv.writer(outfile)
        cols = ['genome'] + [t.__name__ for t in transforms]
        writer.writerow(cols)
        for genome in genomes:
            stats = []
            for transform in transforms:
                print('Applying {0} to {1}',transform.__name__, str(genome))
                stats.append(transform(genome)) # should output some stat for us to log
            writer.writerow([str(genome)] + stats)

In [49]:
g_a = {'name': 2, 2: 3}

In [50]:
apply(['sadsdfdf', 'adsfdf'], [lambda x: len(x), lambda y: len(y)**2 ])

Applying {0} to {1} <lambda> sadsdfdf
Applying {0} to {1} <lambda> sadsdfdf
Applying {0} to {1} <lambda> adsfdf
Applying {0} to {1} <lambda> adsfdf


In [25]:
len('sdfadf')

6

In [26]:
str({'dsf':4})

"{'dsf': 4}"

In [41]:
for i in [0,1]:
    k=2
print(k)

2


In [2]:
b = b'sdadg'

In [3]:
b += b'rewionf kfds'

In [10]:
with x = 4*12:
    print(x)

SyntaxError: invalid syntax (<ipython-input-10-9cc78994e88a>, line 1)

In [13]:
with (return 4*12) as x:
    print(x)

SyntaxError: invalid syntax (<ipython-input-13-5a004697d880>, line 1)